# Automatic differentiation



**! Caution with the functions np.sort, np.where, np.stack, np.broadcast_to !**
* Problem : the arguments are silently cast to np.ndarray, loosing autodiff information.
* Solution : use similarly named replacements from the spad library, which also apply to np.ndarray.

**! Caution with numpy scalars !**
* Problem. In an expression 'a+b' where the l.h.s is a numpy scalar, and the r.h.s an autodiff type of size (,), the r.h.s is silently cast loosing autodiff information.
* Solution : apply 'toarray' to any variable which may be a numpy scalar (in that situation)

In [37]:
import sys; sys.path.append("..") # Allow import from parent directory
from NumericalSchemes import SparseAutomaticDifferentiation as spad

In [38]:
import numpy as np

In [51]:
def toarray(a):
    return a if isinstance(a,np.ndarray) else np.asarray(a)

In [39]:
import importlib
spad = importlib.reload(spad)

arr = np.arange(5)
ad = (5-arr)+spad.identity(arr.shape)

In [41]:
ad/=(1+arr)

In [5]:
ad

spAD(array([5. , 2. , 1. , 0.5, 0.2]), array([[1.        ],
       [0.5       ],
       [0.33333333],
       [0.25      ],
       [0.2       ]]), array([[0],
       [1],
       [2],
       [3],
       [4]]))

In [6]:
sys.getsizeof(arr)

136

In [7]:
np.add(ad,arr)

spAD(array([5. , 3. , 3. , 3.5, 4.2]), array([[1.        ],
       [0.5       ],
       [0.33333333],
       [0.25      ],
       [0.2       ]]), array([[0],
       [1],
       [2],
       [3],
       [4]]))

In [8]:
ad[True]

spAD(array([[5. , 2. , 1. , 0.5, 0.2]]), array([[[1.        ],
        [0.5       ],
        [0.33333333],
        [0.25      ],
        [0.2       ]]]), array([[[0],
        [1],
        [2],
        [3],
        [4]]]))

# Sparse automatic differentiation

The sparse automatic differentiation class inherits from np.ndarray.

In [9]:
#arr = np.ones((2,3))
arr = np.arange(5)
ad = arr+spad.identity(arr.shape)

Elementary properties are inherited

In [10]:
ad.ndim,ad.shape,ad.size,len(ad)

(1, (5,), 5, 5)

However, internal data includes coefficient and index arrays, with one additional dimension.

In [11]:
ad.size_ad

1

Left and right multiplication, addition, substraction, divition, work as usual.

In [12]:
arr + 2*ad

spAD(array([ 0.,  3.,  6.,  9., 12.]), array([[2.],
       [2.],
       [2.],
       [2.],
       [2.]]), array([[0],
       [1],
       [2],
       [3],
       [4]]))

A number of elementary mathematical functions are implemented.

In [13]:
np.sqrt(1+ad)

spAD(array([1.        , 1.41421356, 1.73205081, 2.        , 2.23606798]), array([[0.5       ],
       [0.35355339],
       [0.28867513],
       [0.25      ],
       [0.2236068 ]]), array([[0],
       [1],
       [2],
       [3],
       [4]]))

In [14]:
np.abs(ad-2)

spAD(array([2., 1., 0., 1., 2.]), array([[-1.],
       [-1.],
       [ 0.],
       [ 1.],
       [ 1.]]), array([[0],
       [1],
       [2],
       [3],
       [4]]))

Comparison operators return an ndarray, as well as integer valued functions.

In [15]:
ad <= (-1+arr)

array([False, False, False, False, False])

In [16]:
np.floor(ad+0.5)

array([0., 1., 2., 3., 4.])

A base class ndarray can be recovered by requesting a view, or field 'value', or casting to np.array.

In [17]:
ad.view(np.ndarray), ad.value, np.array(ad)

(array([0., 1., 2., 3., 4.]),
 array([0., 1., 2., 3., 4.]),
 array([0., 1., 2., 3., 4.]))

maximum and minimum work as well

In [18]:
np.maximum(arr,ad)

spAD(array([0., 1., 2., 3., 4.]), array([[1.],
       [1.],
       [1.],
       [1.],
       [1.]]), array([[0],
       [1],
       [2],
       [3],
       [4]]))

### Caveats

**! Caution with the np.sort function (does nothing) !**
Calling it on a spAD type does nothing. Use the spad.sort function instead.

In [19]:
spad.sort(ad) # Correct
# np.sort(ad) # Viciously, does nothing

spAD(array([0., 1., 2., 3., 4.]), array([[1.],
       [1.],
       [1.],
       [1.],
       [1.]]), array([[0],
       [1],
       [2],
       [3],
       [4]]))

**! Caution with np.where, np.stack, np.broadcast_to (silently cast to base class) !**
Some other functions numpy functions will cast their arguments to the base class.
, are not numpy universal functions, and their variant from the spad library needs to be called.

In [20]:
spad.where(ad<arr,ad,arr) # Correct
#np.where(ad<arr,ad,arr) # casts to ndarray

spAD(array([0., 1., 2., 3., 4.]), array([[0.],
       [0.],
       [0.],
       [0.],
       [0.]]), array([[0],
       [0],
       [0],
       [0],
       [0]]))

In [21]:
a,b = spad.identity((2,2))
spad.stack((a,b))
#np.stack((a,b)) # casts to ndarray

spAD(array([[0., 0.],
       [0., 0.]]), array([[[1.],
        [1.]],

       [[1.],
        [1.]]]), array([[[0],
        [1]],

       [[2],
        [3]]]))

**Non-silent failures:** np.reshape.

**! Caution with single element arrays !**

In [22]:
spad = importlib.reload(spad)

In [65]:
a=spad.identity((2,))
b=np.ones((2,))
#print("Correct",a[0]-b[0])
print("Correct ",toarray(b[0])-a[0])
print("Correct  ",toarray(b[0])-toarray(a[0]))
print("Correct", spad.spAD(b[0])-a[0])
print("Correct  ",b[[0]]-a[[0]])
print("Incorrect ! ",b[0]-a[0])

Warning, spAD scalars may get silently cast to np.float64 if appearing on r.h.s of an operand
Correct  spAD(array(1.), array([-1.]), array([0]))
Warning, spAD scalars may get silently cast to np.float64 if appearing on r.h.s of an operand
Correct   spAD(array(1.), array([-1.]), array([0]))
Correct   spAD(array([1.]), array([[-1.]]), array([[0]]))
Warning, spAD scalars may get silently cast to np.float64 if appearing on r.h.s of an operand
Correct spAD(array(1.), array([-1.]), array([0]))
Warning, spAD scalars may get silently cast to np.float64 if appearing on r.h.s of an operand
Incorrect !  1.0


In [64]:
float(spad.identity((2,2))[0,0])

0.0

In [ ]:
np.array(1).shape

### Tested and working
np.take_along_axis, max, min, sum

In [ ]:
ad.max(axis=0)

In [ ]:
np.min(ad,axis=0)

In [ ]:
np.sum(ad,axis=0)